https://github.com/mjpyeon/wavenet-classifier/blob/master/WaveNetClassifier.py

Dataset is the raw RADAR data that has not been reshaped into chirps.

## Notebook setup

Allow editing of modules using editor (auto reloading)

In [2]:
# Needed to allow editing using PyCharm etc
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The following cell is needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [0]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks\\wavenet':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    
DATA_PATH = BASE_PATH + 'data/processed/wavenet/raw_reshaped/'
MODEL_PATH = BASE_PATH + 'models/wavenet/raw_reshaped/test_3/'
    
from src.data import load_data
from src.visualization import multiple_plots, visualize, plot_confusion_matrix
from src.features import make_spectrograms, process_labels, make_directory
from src.models.dilated_CNN.wavenet import WaveNetClassifier

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Activation, Dropout, Add, TimeDistributed, Multiply, Conv1D, Conv2D, MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D
from keras.models import Model, Sequential, load_model
from keras import backend as K
from keras import metrics
from keras import optimizers
from keras.callbacks import History, ModelCheckpoint
from keras.models import Sequential
import time

In [0]:
# Load in data dictionary.
# This does not load in any actual data,
# just the dictionary with the names of the files and their associated labels
with open(DATA_PATH + "index.pkl", "rb") as file:
    data = pickle.load(file)

In [0]:
# Remove user C as this user is reserved for the test set
try:
    del data["C"]
except KeyError:
    print ("Key 'C' not found")

In [0]:
def convert_label_to_int(label):
    if label == "walking":
        return 0
    if label == "pushing":
        return 1
    if label == "sitting":
        return 2
    if label == "pulling":
        return 3
    if label == "circling":
        return 4
    if label == "clapping":
        return 5
    if label == "bending":
        return 6

In [0]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
labels = {}
partition = {'train':[], 'validation':[]} # contains list of training and validation ID's
validation_user = "F"
for user_letter, actions in data.items():
    for action, results in actions.items():
        for result in results:
            for row in result:
                if user_letter == validation_user:
                    partition["validation"].append(row)
                    labels[row] = convert_label_to_int(action)

                else:
                    partition["train"].append(row)
                    labels[row] = convert_label_to_int(action)

In [0]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(3000), n_channels=1,
                 n_classes=7, shuffle=True, data_directory='data/'):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.data_directory = data_directory
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
#         start_time = time.time()

        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
#         time_for_getitem = (time.time() - start_time)
#         print(time_for_getitem)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
#         X = np.empty((self.batch_size, *self.dim))

        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample

#             X[i,] = np.expand_dims(abs(np.load(self.data_directory + ID)), axis=1)
            X[i,] = abs(np.load(self.data_directory + ID))
            # Store class
            y[i] = self.labels[ID]

        return X, to_categorical(y, num_classes=self.n_classes)

In [0]:
target_names = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
nb_classes = len(target_names)

In [0]:
n_filters = 128
dilation_depth = 18
activation = 'softmax'
scale_ratio = 1
kernel_size = 2
pool_size_1 = 4
pool_size_2 = 8
batch_size = 4
epochs = 200

In [0]:
# Parameters
params = {'dim': (3000, 128),
          'batch_size': batch_size,
          'n_classes': nb_classes,
          'shuffle': True,
          'data_directory': DATA_PATH}
# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [0]:
wnc = WaveNetClassifier((3000,128), (7,), kernel_size = kernel_size,
                        dilation_depth = dilation_depth, n_filters = n_filters,
                        pool_size_1=pool_size_1, pool_size_2=pool_size_2)

In [0]:
model = wnc.get_model()

In [0]:
load_weights = False
weights_path = MODEL_PATH + "epoch-33-val_acc-0.14.hdf5"

start_epoch = 0
if load_weights:
    model = load_model(weights_path)
#     model.load_weights(weights_path)
    last_epoch = weights_path.split("-")[-3]
    start_epoch = int(last_epoch)

In [0]:
if not load_weights:
    model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
checkpoint = ModelCheckpoint(MODEL_PATH + "epoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5",
                             monitor='val_acc', verbose=0, save_best_only=False,
                             save_weights_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

In [0]:
# Train model on dataset
history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=9,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    initial_epoch=start_epoch)

In [0]:
visualize.plot_train_test_acc(history=history)